In [ ]:
!pip install streamlit pyngrok


In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv("/content/merged_hourly_with_pv.csv", parse_dates=["timestamp"])
monthly_bills = pd.read_csv("/content/monthly_bills.csv")

df = df.set_index("timestamp")


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

st.set_page_config(page_title="123W93Street Solar Dashboard – 2023", layout="wide")

# Load data
df = pd.read_csv("merged_hourly_with_pv.csv", parse_dates=["timestamp"])
monthly_bills = pd.read_csv("monthly_bills.csv")
weather = pd.read_csv("nsrdb_weather_nyc_2023_hourly.csv", parse_dates=["time"])

df = df.set_index("timestamp")
weather['season'] = weather['time'].dt.month % 12 // 3 + 1
season_map = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
weather['season'] = weather['season'].map(season_map)
weather['hour'] = weather['time'].dt.hour
weather['date'] = weather['time'].dt.date

st.title("123W93Street Solar Savings Dashboard – 2023")

tab0, tab1, tab2, tab3, tab4, tab5 = st.tabs([
    "Financial Scenario Tool",
    "Key Performance Indicators",
    "Billing Insights",
    "PV Generation Insights",
    "Weather & Consumption Insights",
    "Tariff & Assumptions"
])
# ---- TAB 0 ----
with tab0:
    st.header("Financial Modeling – 25 Year Solar Scenario")

    st.markdown("Adjust the assumptions below to simulate long-term savings and ROI.")

    # Input parameters
    colA, colB, colC = st.columns(3)
    nameplate_kw = colA.number_input("Nameplate Solar Capacity (kW)", value=100)
    specific_yield = colB.number_input("Specific Yield (kWh/kWp)", value=1200)
    vder_rate = colC.number_input("VDER Rate ($/kWh)", value=0.15)

    colD, colE, colF = st.columns(3)
    cost_per_watt = colD.number_input("Cost per Watt ($)", value=3.0)
    degradation = colE.number_input("Annual Degradation Rate (%)", value=0.5)
    maintenance_cost = colF.number_input("Maintenance Cost ($/kW/year)", value=15)

    inflation = st.number_input("Annual Inflation Rate (%)", value=3.0)

    # Constants
    capital_cost = nameplate_kw * cost_per_watt * 1000  # Convert to dollars

    # Simulate 25 years
    years = list(range(1, 26))
    energy_output = []
    revenue = []
    expense = []
    net_income = []
    cumulative_cashflow = []

    for year in years:
        degradation_factor = (1 - degradation / 100) ** year
        inflation_factor = (1 + inflation / 100) ** year

        annual_energy = nameplate_kw * specific_yield * degradation_factor
        annual_revenue = annual_energy * vder_rate * inflation_factor
        annual_maintenance = maintenance_cost * nameplate_kw * inflation_factor
        annual_net = annual_revenue - annual_maintenance

        energy_output.append(annual_energy)
        revenue.append(annual_revenue)
        expense.append(annual_maintenance)
        net_income.append(annual_net)
        cumulative_cashflow.append(sum(net_income))
    import plotly.graph_objects as go

    # Plot financial trends
    st.subheader("25-Year Revenue, Expenses, and Net Income Forecast")
    fig_financials = go.Figure()
    fig_financials.add_trace(go.Scatter(x=years, y=revenue, name="Revenue", mode='lines+markers'))
    fig_financials.add_trace(go.Scatter(x=years, y=expense, name="Maintenance Cost", mode='lines+markers'))
    fig_financials.add_trace(go.Scatter(x=years, y=net_income, name="Net Income", mode='lines+markers'))
    fig_financials.update_layout(
        xaxis_title="Year",
        yaxis_title="USD",
        title="Annual Financial Projections",
        legend_title="Category",
        template="plotly_white"
    )
    st.plotly_chart(fig_financials, use_container_width=True)

    # ROI and Payback Summary
    st.subheader("Investment Summary")

    payback_year = next((year for year, cash in zip(years, cumulative_cashflow) if cash >= capital_cost), None)
    roi_25yr = (cumulative_cashflow[-1] - capital_cost) / capital_cost * 100

    col1, col2, col3 = st.columns(3)
    col1.metric("Initial Capital Cost", f"${capital_cost:,.0f}")
    col2.metric("25-Year ROI", f"{roi_25yr:.2f}%")
    col3.metric("Estimated Payback Year", f"Year {payback_year}" if payback_year else "Beyond 25 years")

    # Optional: Cumulative Profit Plot
    st.subheader("Cumulative Net Savings")
    fig_cum = go.Figure()
    fig_cum.add_trace(go.Scatter(x=years, y=[c - capital_cost for c in cumulative_cashflow],
                                 mode='lines+markers', name='Cumulative Net Profit'))
    fig_cum.update_layout(
        xaxis_title="Year",
        yaxis_title="Cumulative Profit ($)",
        title="Cumulative Profit vs Initial Investment",
        template="plotly_white"
    )
    st.plotly_chart(fig_cum, use_container_width=True)


# ---- TAB 1 ----
with tab1:
    st.header("Annual KPIs – 2023")

    total_pv_generation = df['pv_ac_kw'].sum()
    total_baseline_kwh = df['raw_MM_Wh'].sum() / 1000
    pv_offset_percent = (total_pv_generation / total_baseline_kwh) * 100

    total_revenue = monthly_bills['usd_saved'].sum()
    total_baseline_bill = monthly_bills['baseline_bill_usd'].sum()
    total_pv_bill = monthly_bills['bill_with_pv_usd'].sum()
    percent_savings = (total_revenue / total_baseline_bill) * 100
    average_monthly_baseline = total_baseline_bill / 12
    average_monthly_pv = total_pv_bill / 12

    col1, col2, col3 = st.columns(3)
    col1.metric("Total PV Generation (kWh)", f"{total_pv_generation:,.0f}")
    col2.metric("Total Building Load (kWh)", f"{total_baseline_kwh:,.0f}")
    col3.metric("PV Offset (%)", f"{pv_offset_percent:.2f}%")

    col4, col5, col6 = st.columns(3)
    col4.metric("Total Revenue from PV ($)", f"${total_revenue:,.0f}")
    col5.metric("Total % Savings", f"{percent_savings:.2f}%")
    col6.metric("Avg Monthly Bill (after PV)", f"${average_monthly_pv:,.0f}")

# ---- TAB 2 ----
with tab2:
    st.header("Billing and Savings Analysis – 2023")

    fig, ax = plt.subplots(figsize=(12,6))
    month_index = monthly_bills["month"]

    ax.bar(month_index - 0.2, monthly_bills["baseline_bill_usd"], width=0.4, label="Baseline Bill")
    ax.bar(month_index + 0.2, monthly_bills["bill_with_pv_usd"], width=0.4, label="Bill After PV")

    ax.set_xlabel("Month")
    ax.set_ylabel("Bill ($)")
    ax.set_title("Monthly Electricity Bills: Baseline vs With PV (2023)")
    ax.set_xticks(month_index)
    ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    ax.legend()
    ax.grid(True, axis='y')
    st.pyplot(fig)

    st.subheader("Monthly Cost Savings (USD)")
    fig2, ax2 = plt.subplots(figsize=(12,5))
    ax2.bar(month_index, monthly_bills["usd_saved"], color="green")
    ax2.set_xlabel("Month")
    ax2.set_ylabel("USD Saved")
    ax2.set_title("Monthly Savings After PV Installation (2023)")
    ax2.set_xticks(month_index)
    ax2.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    ax2.grid(True, axis="y")
    st.pyplot(fig2)

    st.subheader("Cumulative Revenue Savings (USD)")
    monthly_bills["cumulative_usd_saved"] = monthly_bills["usd_saved"].cumsum()
    fig3, ax3 = plt.subplots(figsize=(12,5))
    ax3.plot(month_index, monthly_bills["cumulative_usd_saved"], marker="o", linewidth=2, color="blue")
    ax3.set_xlabel("Month")
    ax3.set_ylabel("Cumulative USD Saved")
    ax3.set_title("Cumulative PV Revenue Savings Over 2023")
    ax3.grid(True)
    st.pyplot(fig3)

    st.subheader("Interactive Month/Season Summary")

    selected_view = st.radio(
        "Select a Time Period:",
        options=["January", "February", "March", "April", "May", "June",
                 "July", "August", "September", "October", "November", "December",
                 "Winter", "Spring", "Summer", "Fall"],
        horizontal=True
    )

    month_mapping = {
        "January": 1, "February": 2, "March": 3, "April": 4,
        "May": 5, "June": 6, "July": 7, "August": 8,
        "September": 9, "October": 10, "November": 11, "December": 12
    }

    season_mapping = {
        "Winter": [12, 1, 2],
        "Spring": [3, 4, 5],
        "Summer": [6, 7, 8],
        "Fall": [9, 10, 11]
    }

    if selected_view in month_mapping:
        months = [month_mapping[selected_view]]
    else:
        months = season_mapping[selected_view]

    # Filter data
    filtered_df = df[df.index.month.isin(months)]
    filtered_bills = monthly_bills[monthly_bills["month"].isin(months)]
    filtered_weather = weather[weather["time"].dt.month.isin(months)]

    # KPIs
    total_consumption_kwh = filtered_df["raw_MM_Wh"].sum() / 1000
    total_pv_kwh = filtered_df["pv_ac_kw"].sum()
    baseline_bill = filtered_bills["baseline_bill_usd"].sum()
    pv_bill = filtered_bills["bill_with_pv_usd"].sum()
    total_savings = baseline_bill - pv_bill
    percent_savings = (total_savings / baseline_bill) * 100 if baseline_bill > 0 else 0
    avg_temp = filtered_weather["temp_air"].mean()
    avg_ghi = filtered_weather["ghi"].mean()

    st.markdown("### Summary for {}".format(selected_view))

    colm1, colm2, colm3 = st.columns(3)
    colm1.metric("Total Consumption (kWh)", f"{total_consumption_kwh:,.0f}")
    colm2.metric("Total PV Generation (kWh)", f"{total_pv_kwh:,.0f}")
    colm3.metric("PV Offset (%)", f"{(total_pv_kwh/total_consumption_kwh)*100:.2f}%" if total_consumption_kwh > 0 else "N/A")

    colm4, colm5, colm6 = st.columns(3)
    colm4.metric("Baseline Bill ($)", f"${baseline_bill:,.0f}")
    colm5.metric("Bill After PV ($)", f"${pv_bill:,.0f}")
    colm6.metric("Total Savings (%)", f"{percent_savings:.2f}%")

    colm7, colm8 = st.columns(2)
    colm7.metric("Avg Temperature (°C)", f"{avg_temp:.1f}")
    colm8.metric("Avg Solar Radiation (GHI)", f"{avg_ghi:.0f} W/m²")

    # ==== New Visualizations ====

    if not filtered_df.empty:
        st.subheader("Hourly Electricity Consumption and PV Output")

        fig_hourly = px.line(
            filtered_df.reset_index(),
            x="timestamp",
            y=["raw_MM_Wh", "pv_ac_kw"],
            labels={"timestamp": "Time", "value": "Energy (W / kW)", "variable": "Source"},
            title=f"Hourly Consumption vs PV Output - {selected_view}"
        )
        st.plotly_chart(fig_hourly, use_container_width=True)

        st.subheader("Distribution of Hourly Consumption (kWh)")

        fig_hist = px.histogram(
            filtered_df,
            x=filtered_df["raw_MM_Wh"] / 1000,
            nbins=30,
            labels={"x": "Hourly Consumption (kWh)"},
            title=f"Hourly Consumption Distribution - {selected_view}"
        )
        st.plotly_chart(fig_hist, use_container_width=True)

    else:
        st.warning("No data available for this selection.")



# ---- TAB 3 ----
with tab3:
    st.header("PV Generation Insights – 2023")

    df['month'] = df.index.month
    fig4, ax4 = plt.subplots(figsize=(12,5))
    monthly_boxes = [df.loc[df['month'] == m, 'pv_ac_normalized'].dropna() for m in range(1, 13)]

    ax4.boxplot(monthly_boxes, showfliers=False)
    ax4.set_xticks(range(1,13))
    ax4.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    ax4.set_xlabel('Month')
    ax4.set_ylabel('Normalized PV Output')
    ax4.set_title('Monthly Variation of Normalized PV Output (2023)')
    ax4.grid(True)
    st.pyplot(fig4)

    df['hour'] = df.index.hour
    fig5, ax5 = plt.subplots(figsize=(12,5))
    hourly_boxes = [df.loc[df['hour'] == h, 'pv_ac_normalized'].dropna() for h in range(24)]

    ax5.boxplot(hourly_boxes, showfliers=False)
    ax5.set_xticks(range(1,25))
    ax5.set_xticklabels([str(h) for h in range(24)])
    ax5.set_xlabel('Hour of Day')
    ax5.set_ylabel('Normalized PV Output')
    ax5.set_title('Hourly Variation of Normalized PV Output (2023)')
    ax5.grid(True)
    st.pyplot(fig5)

    st.subheader("PV Output Heatmap (Hourly PV Output)")
    pv_heatmap = df.reset_index().pivot_table(index=df.reset_index()['timestamp'].dt.date, columns='hour', values='pv_ac_kw', aggfunc='mean')
    fig14 = px.imshow(
        pv_heatmap.fillna(0),
        labels={"x": "Hour of Day", "y": "Date", "color": "PV Output (kW)"},
        aspect="auto",
        title="PV Output Heatmap by Date and Hour (2023)"
    )
    st.plotly_chart(fig14, use_container_width=True)


# ---- TAB 4 ----
with tab4:
    st.header("Weather & Load Insights – 2023")

    # Merge load and weather data
    merged_weather_load = pd.merge(df.reset_index(), weather, left_on="timestamp", right_on="time", how="inner")
    merged_weather_load['load_kwh'] = merged_weather_load['raw_MM_Wh'] / 1000
    merged_weather_load["hour_x"] = pd.to_numeric(merged_weather_load["hour_x"], errors="coerce")

    # Section 1: Scatter plot – Hourly Load vs Temperature
    st.subheader("Hourly Electricity Consumption vs Temperature")
    fig1 = px.scatter(
        merged_weather_load,
        x="temp_air_y",
        y="load_kwh",
        color="hour_x",  # color by hour
        color_continuous_scale="Viridis",
        trendline="ols",
        labels={"temp_air_y": "Temperature (°C)", "load_kwh": "Load (kWh)", "hour_x": "Hour"},
        title="Hourly Load vs Temperature (Colored by Hour of Day)"
    )
    st.plotly_chart(fig1, use_container_width=True)

    # Section 2: Avg Load by Temperature Bin
    st.subheader("Average Load by Temperature Bin (2°C Steps)")
    merged_weather_load["temp_bin"] = (merged_weather_load["temp_air_y"] // 2) * 2
    temp_avg_load = merged_weather_load.groupby("temp_bin")["load_kwh"].mean().reset_index()
    fig2 = px.line(
        temp_avg_load,
        x="temp_bin",
        y="load_kwh",
        markers=True,
        labels={"temp_bin": "Temperature Bin (°C)", "load_kwh": "Avg Load (kWh)"},
        title="Avg Load per Temperature Bin (2°C steps)"
    )
    st.plotly_chart(fig2, use_container_width=True)

    # Section 3: Season Selector for Avg Weather
    st.subheader("Average Hourly Weather Patterns by Season")
    season_options = ["Winter", "Spring", "Summer", "Fall"]
    selected_season = st.radio("Select a Season:", options=season_options, horizontal=True)

    season_months = {
        "Winter": [12, 1, 2],
        "Spring": [3, 4, 5],
        "Summer": [6, 7, 8],
        "Fall": [9, 10, 11],
    }
    filtered_weather = weather[weather["time"].dt.month.isin(season_months[selected_season])]
    filtered_weather["hour"] = filtered_weather["time"].dt.hour

    avg_hourly = filtered_weather.groupby("hour")[["ghi", "dhi", "dni", "temp_air"]].mean().reset_index()

    col1, col2 = st.columns(2)

    with col1:
        fig3 = px.line(
            avg_hourly,
            x="hour",
            y=["ghi", "dni", "dhi"],
            labels={"value": "W/m²", "hour": "Hour"},
            title=f"Avg Solar Irradiance – {selected_season}"
        )
        st.plotly_chart(fig3, use_container_width=True)

    with col2:
        fig4 = px.line(
            avg_hourly,
            x="hour",
            y="temp_air",
            labels={"temp_air": "°C", "hour": "Hour"},
            title=f"Avg Ambient Temperature – {selected_season}"
        )
        st.plotly_chart(fig4, use_container_width=True)

    # Section 4: Heatmap – GHI by Hour and Day
    st.subheader("Solar Irradiance Heatmap")
    weather["date"] = weather["time"].dt.date
    ghi_heatmap_df = weather.pivot_table(index="date", columns="hour", values="ghi", aggfunc="mean").fillna(0)
    fig5 = px.imshow(
        ghi_heatmap_df,
        labels={"x": "Hour", "y": "Date", "color": "GHI (W/m²)"},
        title="GHI Heatmap by Hour and Date (2023)",
        aspect="auto"
    )
    st.plotly_chart(fig5, use_container_width=True)

    # Section 5: Significant Days
    st.subheader("Significant Solar Days in 2023")
    daily_ghi = weather.groupby("date").agg({
        "ghi": ["mean", "std"],
        "temp_air": ["min", "max"]
    }).reset_index()
    daily_ghi.columns = ["date", "ghi_mean", "ghi_std", "temp_min", "temp_max"]
    daily_ghi["temp_range"] = daily_ghi["temp_max"] - daily_ghi["temp_min"]

    col3, col4 = st.columns(2)

    with col3:
        st.markdown("**☀️ Peak Solar Days**")
        st.dataframe(daily_ghi.nlargest(3, "ghi_mean")[["date", "ghi_mean"]])

        st.markdown("**🌫️ Low Solar Days**")
        st.dataframe(daily_ghi.nsmallest(3, "ghi_mean")[["date", "ghi_mean"]])

    with col4:
        st.markdown("**🌡️ High Temp Swings**")
        st.dataframe(daily_ghi.nlargest(3, "temp_range")[["date", "temp_range"]])

        st.markdown("**🌤️ High GHI Variability**")
        st.dataframe(daily_ghi.nlargest(3, "ghi_std")[["date", "ghi_std"]])


# ---- TAB 5 ----
with tab5:
    st.header("Tariff and Billing Assumptions")

    st.subheader("SC9.1 Tariff Breakdown (2023)")
    st.markdown("""
    - Customer Charge: $71.00 per month
    - Delivery Charges: $0.0166 per kWh + Rule 26 Adjustments
    - Demand Charge:
        - Summer (June–August): $42.80 per peak kW
        - Non-Summer: $33.50 per peak kW
    - Supply Charges: $0.08195 per kWh + Capacity Charges
    - Taxes: GRT Surcharges and Sales Tax apply
    """)

    st.subheader("Billing Assumptions")
    st.markdown("""
    - PV output offsets building load before billing
    - No negative net load (no battery storage assumed)
    - Summer months are June, July, August
    """)




Overwriting app.py


In [ ]:
!ngrok config add-authtoken 2vPbi8ZSfsUB0ITjrnHASQlUHtE_2nVgMVJ29Ea5nM9erq2X9

import os
import time
from pyngrok import ngrok

os.system("pkill streamlit")
os.system("streamlit run app.py &")
time.sleep(10)

public_url = ngrok.connect(8501)
print("Your Streamlit app is live at:", public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Your Streamlit app is live at: NgrokTunnel: "https://50ae-34-148-173-30.ngrok-free.app" -> "http://localhost:8501"
